# $\alpha$=0.1

In [6]:
import numpy as np
import torch
import torchvision.transforms as transforms              
from torchvision.datasets import CIFAR100      
from torch.utils.data import DataLoader
import timm
import detectors
from src.saps import split_data_set, saps_scores, saps_classification, eval_aps
from src.temperature_scaling import ModelWithTemperature

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet18_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR100
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# error rate
alpha = 0.1
lambda_ = 0.13
num_runs = 10

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.116, ECE: 0.050
Optimal temperature: 4.704
After temperature - NLL: 3.091, ECE: 0.664
SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.4093396335840226
Average Prediction Set Size After APS in runs 1: 3.3236
Average Coverage Rate in runs 1: 0.9

Running experiment 2/10...
t_cal = 0.41234659552574204
Average Prediction Set Size After APS in runs 2: 3.3524
Average Coverage Rate in runs 2: 0.9028

Running experiment 3/10...
t_cal = 0.4214461654424675
Average Prediction Set Size After APS in runs 3: 3.4328
Average Coverage Rate in runs 3: 0.9044

Running experiment 4/10...
t_cal = 0.4174623161554342
Average Prediction Set Size After APS in runs 4: 3.3994
Average Coverage Rate in runs 4: 0.906

Running experiment 5/10...
t_cal = 0.4059896081686025
Average Prediction Set Size After APS in runs 5: 3.3082
Average Coverage Rate in runs 5: 0.8992

Running experiment 6/10...
t_cal = 0.40398762226104784
Average Pred

# $\alpha$=0.2

In [11]:
# error rate
alpha = 0.2
lambda_ = 0.5

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.32845089435577424
Average Prediction Set Size After APS in runs 1: 1.4278
Average Coverage Rate in runs 1: 0.7962

Running experiment 2/10...
t_cal = 0.3505808472633365
Average Prediction Set Size After APS in runs 2: 1.4812
Average Coverage Rate in runs 2: 0.8038

Running experiment 3/10...
t_cal = 0.34287389516830463
Average Prediction Set Size After APS in runs 3: 1.484
Average Coverage Rate in runs 3: 0.8068

Running experiment 4/10...
t_cal = 0.33004751801490795
Average Prediction Set Size After APS in runs 4: 1.4598
Average Coverage Rate in runs 4: 0.7994

Running experiment 5/10...
t_cal = 0.35082234144210916
Average Prediction Set Size After APS in runs 5: 1.507
Average Coverage Rate in runs 5: 0.8082

Running experiment 6/10...
t_cal = 0.34268652200698885
Average Prediction Set Size After APS in runs 6: 1.4756
Average Coverage Rate in runs 6: 0.7984

Running experiment 7/10...
t_cal = 0.3579909265041353
Average 

# $\alpha$=0.05

In [16]:
# error rate
alpha = 0.05
lambda_ = 0.05

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("SAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = saps_scores(model, calib_loader, alpha, lambda_, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = saps_classification(model, test_loader, t_cal, lambda_, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

SAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.46000394523143806
Average Prediction Set Size After APS in runs 1: 8.0706
Average Coverage Rate in runs 1: 0.9528

Running experiment 2/10...
t_cal = 0.4507408156991006
Average Prediction Set Size After APS in runs 2: 7.8622
Average Coverage Rate in runs 2: 0.9494

Running experiment 3/10...
t_cal = 0.4558900415897371
Average Prediction Set Size After APS in runs 3: 8.0288
Average Coverage Rate in runs 3: 0.9506

Running experiment 4/10...
t_cal = 0.4655338704586031
Average Prediction Set Size After APS in runs 4: 8.1788
Average Coverage Rate in runs 4: 0.9536

Running experiment 5/10...
t_cal = 0.4715491607785231
Average Prediction Set Size After APS in runs 5: 8.2994
Average Coverage Rate in runs 5: 0.9544

Running experiment 6/10...
t_cal = 0.45131094902753854
Average Prediction Set Size After APS in runs 6: 7.9102
Average Coverage Rate in runs 6: 0.9496

Running experiment 7/10...
t_cal = 0.46026643067598344
Average 

# Result
  
$\alpha$=0.1
- Final Average **Prediction Set Size： 3.35**
- Final Average **Coverage: 90.14%**  

$\alpha$=0.2
- Final Average **Prediction Set Size： 1.46**
- Final Average **Coverage: 79.99%**  

$\alpha$=0.05
- Final Average **Prediction Set Size：7.86**
- Final Average **Coverage: 94.93%**